<a href="https://colab.research.google.com/github/AnnuPrasad/Chatbot_bengali/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import nltk

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

import json

import pickle

import numpy as np

from keras.models import Sequential

from keras.layers import Dense, Activation, Dropout

from keras.optimizers import SGD

import random

In [39]:
# Sample dataset (you can replace this with your Bengali dataset)
data_1 = open('data.json').read()

data = json.loads(data_1)

In [40]:
!pip install SpeechRecognition

In [41]:
!pip install pydub

In [45]:
import os
from pydub import AudioSegment
from nltk.corpus import stopwords
import speech_recognition as sr


for filename in os.listdir("/content/audios"):
    if not filename.endswith(".wav"):
        print(f"File '{filename}' is not in WAV format.")

# Initialize the recognizer
recognizer = sr.Recognizer()

# Define a function to transcribe audio files
import os
from pydub import AudioSegment
import speech_recognition as sr

# Initialize the recognizer
recognizer = sr.Recognizer()

def transcribe_audio(audio_folder):
    transcribed_texts = []  # List to store transcribed texts
    for filename in os.listdir(audio_folder):
        if filename.endswith(".wav"):  # Check if the file is a WAV file
            audio_file_path = os.path.join(audio_folder, filename)
            with sr.AudioFile(audio_file_path) as source:
                audio_data = recognizer.record(source)
    return transcribed_texts


audio_folder_path = "/content/audios"
transcribed_texts = transcribe_audio(audio_folder_path)


# Replace sample_data with the transcribed text
data_1 = open("data.json").read()

data = json.loads(data_1)

# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('bengali'))


# # Tokenize, lemmatize, and remove stopwords
# tokenized_data = []
# for sentence in data:
#     lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize tokens
#     filtered_tokens = [token for token in lemmatized_tokens if token not in stop_words]  # Remove stop words
#     tokenized_data.append(filtered_tokens)

# # Remove duplicate words
# for i in range(len(tokenized_data)):
#     tokenized_data[i] = list(set(tokenized_data[i]))

# print(tokenized_data)

File '.ipynb_checkpoints' is not in WAV format.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
!pip install classes

In [62]:
!pip install bnlp

  Preparing metadata (setup.py) ... done
  Created wheel for bnlp: filename=bnlp-0.8-py3-none-any.whl size=32803 sha256=7605d254774b3032de34026d916b02903f170757f73f8c948f38b5382075ca86
  Stored in directory: /root/.cache/pip/wheels/64/c7/7e/272284eb8c800fbb5306bdf7904ae9f99a64812fc615f471b2
Successfully built bnlp


In [70]:
import numpy as np
import random
from nltk.tokenize import word_tokenize

# Define your classes for Bengali language
classes = ["politics", "sports", "entertainment", "technology", "health", "education"]

# Sample documents (replace with your actual data)
documents = [
    ("খেলাধুলা সম্পর্কে সংবাদ", "sports"),
    ("প্রযুক্তি নিয়ে আলোচনা", "technology"),
    ("সরকারের নীতিমালা সম্পর্কে বিবৃতি", "politics"),
    ("বিনোদনের প্রচ্ছদ", "entertainment"),
    ("স্বাস্থ্য সংক্রান্ত প্রতিবেদন", "health"),
    ("শিক্ষা ও শিক্ষার্থীরা", "education")
]

# Initialize an empty list for training data
training = []

# Create an empty array of zeros for output representation
output_empty = [0] * len(classes)

# Collect all unique words from documents
words = set()
for doc in documents:
    words.update(word_tokenize(doc[0]))

# Iterate over each document for processing
for doc in documents:
    bag = []

    # Tokenize words in the document
    pattern_words = word_tokenize(doc[0])
    pattern_words = [word.lower() for word in pattern_words]

    # Create bag of words representation
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Create output row for classification
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # Append bag of words and output row to training data
    training.append([bag, output_row])

# Shuffle training data
random.shuffle(training)

# Separate features and labels
train_x = np.array([sample[0] for sample in training])
train_y = np.array([sample[1] for sample in training])

# Print shapes for verification
print("Shape of train_x:", train_x.shape)
print("Shape of train_y:", train_y.shape)


Shape of train_x: (6, 17)
Shape of train_y: (6, 6)


In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.legacy import SGD as LegacySGD

# Define the model
model = Sequential()

# Add layers to the model
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Use the legacy optimizer
sgd = LegacySGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train the model
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Save the model
model.save('chatbot_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
2/2 [==============================] - 1s 21ms/step - loss: 1.9768 - accuracy: 0.0000e+00
Epoch 2/200
2/2 [==============================] - 0s 8ms/step - loss: 1.9331 - accuracy: 0.0000e+00
Epoch 3/200
2/2 [==============================] - 0s 8ms/step - loss: 1.8359 - accuracy: 0.0000e+00
Epoch 4/200
2/2 [==============================] - 0s 8ms/step - loss: 1.7753 - accuracy: 0.3333
Epoch 5/200
2/2 [==============================] - 0s 9ms/step - loss: 1.7407 - accuracy: 0.3333
Epoch 6/200
2/2 [==============================] - 0s 9ms/step - loss: 1.7999 - accuracy: 0.3333
Epoch 7/200
2/2 [==============================] - 0s 10ms/step - loss: 1.6167 - accuracy: 0.5000
Epoch 8/200
2/2 [==============================] - 0s 8ms/step - loss: 1.7857 - accuracy: 0.3333
Epoch 9/200
2/2 [==============================] - 0s 8ms/step - loss: 1.5215 - accuracy: 0.5000
Epoch 10/200
2/2 [==============================] - 0s 8ms/step - loss: 1.5702 - accuracy: 0.6667
Epoch 11/200
2/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [74]:
def predict_class(sentence, model):

    p = bow(sentence, words, show_details=False)

    res = model.predict(np.array([p]))[0]

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)

    return_list = []

    for r in results:

        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list

In [1]:
!pip install PyQt5

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QTextEdit, QPushButton, QVBoxLayout, QWidget

# Function to send message and get response
def send():
    msg = EntryBox.toPlainText().strip()
    EntryBox.clear()
    if msg != '':
        ChatLog.setPlainText(ChatLog.toPlainText() + "You: " + msg + '\n\n')
        res = chatbot_response(msg)
        ChatLog.setPlainText(ChatLog.toPlainText() + "Bot: " + res + '\n\n')

# GUI setup
class ChatbotWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Chatbot")
        self.setGeometry(100, 100, 400, 500)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        layout = QVBoxLayout()
        self.central_widget.setLayout(layout)

        self.ChatLog = QTextEdit()
        self.ChatLog.setReadOnly(True)
        layout.addWidget(self.ChatLog)

        self.EntryBox = QTextEdit()
        layout.addWidget(self.EntryBox)

        self.SendButton = QPushButton("Send")
        self.SendButton.clicked.connect(send)
        layout.addWidget(self.SendButton)

app = QApplication(sys.argv)
window = ChatbotWindow()
window.show()
sys.exit(app.exec_())
